In [1]:
import pandas as pd
import numpy as np
import folium
import requests
import json
import pprint
import datetime as dt
import re

In [2]:
room_data = pd.read_csv('room.csv',encoding='CP949')
room_data.head()

,건물명(상호명),건물위치,주택유형구분,세대수(객실수),건축연도
0,NaN,전라북도 전주시 덕진구 고랑동 819-5,다세대주택(3세대),3,19920810
1,NaN,전라북도 전주시 덕진구 금암동 1569-6,다세대주택(3세대),6,19891206
2,NaN,전라북도 전주시 덕진구 금암동 1588-23,다세대주택(3세대),3,19890803
3,휴엔하임 주상복합 아파트,전라북도 전주시 덕진구 금암동 1639-1,"공동주택(아파트), 업무시설(오피스텔), 근린생활시설",30,20161128
4,NaN,전라북도 전주시 덕진구 금암동 525-83,다세대주택,4,19891121


In [3]:
room_data.head()

,건물명(상호명),건물위치,주택유형구분,세대수(객실수),건축연도
0,NaN,전라북도 전주시 덕진구 고랑동 819-5,다세대주택(3세대),3,19920810
1,NaN,전라북도 전주시 덕진구 금암동 1569-6,다세대주택(3세대),6,19891206
2,NaN,전라북도 전주시 덕진구 금암동 1588-23,다세대주택(3세대),3,19890803
3,휴엔하임 주상복합 아파트,전라북도 전주시 덕진구 금암동 1639-1,"공동주택(아파트), 업무시설(오피스텔), 근린생활시설",30,20161128
4,NaN,전라북도 전주시 덕진구 금암동 525-83,다세대주택,4,19891121


In [4]:
room_data.describe()

,세대수(객실수)
count,8734.000000
mean,8.135791
std,11.724445
min,0.000000
25%,3.000000
50%,8.000000
75%,11.000000
max,826.000000


In [5]:
## TODO
## 건물 위치를 참조하여 folium에 좌표 찍기
## 건축 연도를 참조하여 room_data에 신축 / 준신축 / 보통 / 재건축필요 등의 등급으로 분류

# room_data['건물위치']
# key = "7F8A26C5-7C0A-318B-A521-FE1AA35942DB"
# arr = []
# arr = []
# f = open("./loc.txt", 'w')
# f.close()
# import time
# for i in room_data['건물위치']:
#     f = open("./loc.txt", 'a')
#     location = i.strip()
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=PARCEL")
#     if 'result' in response.json()['response']:
#         x= float(response.json()['response']['result']['point']['x'])
#         y = float(response.json()['response']['result']['point']['y'])
#         arr.append([float(x),float(y)])
#         f.write(str(x))
#         f.write(',')
#         f.write(str(y))
#         f.write('\n')
#         f.close()
#     else:
#         print(location, response.json())
#     time.sleep(0.25)

In [6]:
f = open('./loc.txt', 'r')
lines = f.readlines()
map1 = folium.Map(location=[35.83001,127.125001], zoom_start=13)
arr = []
for line in lines:
    x,y = map(float, line.split(','))
    x = round(x,2)
    y = round(y,2)
    if [x,y] not in arr:
        arr.append([x,y])

for content in arr:
    x,y = content
    folium.Marker([y, x], popup='원룸', icon=folium.Icon(color='red',icon='info-sign')).add_to(map1)
    folium.CircleMarker([y, x], radius=30, color='#3186cc', fill_color='#3186cc', popup='원룸').add_to(map1)
map1.save('map1.html')

In [7]:
p = re.compile(r'\d+')
room_data['건축연도'] = room_data['건축연도'].map(lambda x: x if p.search(x) else '0')
room_data['const_year_point'] = 5-(dt.datetime.today().year-room_data['건축연도'].map(lambda x: int(x[:4])))//5

room_data.head(10)

,건물명(상호명),건물위치,주택유형구분,세대수(객실수),건축연도,const_year_point
0,NaN,전라북도 전주시 덕진구 고랑동 819-5,다세대주택(3세대),3,19920810,0
1,NaN,전라북도 전주시 덕진구 금암동 1569-6,다세대주택(3세대),6,19891206,-1
2,NaN,전라북도 전주시 덕진구 금암동 1588-23,다세대주택(3세대),3,19890803,-1
3,휴엔하임 주상복합 아파트,전라북도 전주시 덕진구 금암동 1639-1,"공동주택(아파트), 업무시설(오피스텔), 근린생활시설",30,20161128,4
4,NaN,전라북도 전주시 덕진구 금암동 525-83,다세대주택,4,19891121,-1
5,덕진동1가 1267-32 공동주택,전라북도 전주시 덕진구 덕진동1가 1267-32,공동주택,2,20130131,4
6,NaN,전라북도 전주시 덕진구 덕진동1가 1360-18,다세대주택,2,19891226,-1
7,세움 펠리피아,전라북도 전주시 덕진구 반월동 1231,"아파트,업무시설,근린생활시설",28,20160211,4
8,NaN,전라북도 전주시 덕진구 송천동1가 367,다세대주택,4,19881025,-1
9,NaN,전라북도 전주시 덕진구 여의동 681-1,다세대주택(2세대),2,19911028,-1


In [19]:
from xml.etree.ElementTree import parse
from bs4 import BeautifulSoup

url = "http://openapi.jeonju.go.kr/jeonjubus/openApi/traffic/bus_location2_stopnm_common.do?serviceKey=cjE984BWrGFH3DFA2kLDoNx7j950nWjp6Ajz0CBaEABY5arYHY1C80RNBw6ptmayEV6B0zzpOfAbB73miZ4gnA%3D%3D&searchFld=searchNM&searchNm=%EC%95%88%EA%B3%A8"
response = requests.get(url)
data = BeautifulSoup(response.text, 'xml')
data

<?xml version="1.0" encoding="utf-8"?>
<RFC30><code type="string">000</code><msg type="string">OK</msg><routeList class="array"><list class="object"><reMark type="string"/><searchFld type="string"/><searchNm type="string"/><stopId type="string">30693</stopId><stopKname type="string">BYC</stopKname><stopStandardid type="string">312100693</stopStandardid><stopX type="string">127.00796999</stopX><stopY type="string">35.82071324</stopY></list><list class="object"><reMark type="string"/><searchFld type="string"/><searchNm type="string"/><stopId type="string">30690</stopId><stopKname type="string">BYC</stopKname><stopStandardid type="string">312100690</stopStandardid><stopX type="string">127.00778106</stopX><stopY type="string">35.82043092</stopY></list><list class="object"><reMark type="string"/><searchFld type="string"/><searchNm type="string"/><stopId type="string">30683</stopId><stopKname type="string">CTS전북방송</stopKname><stopStandardid type="string">305100683</stopStandardid><stopX type